In [11]:
import os

In [12]:
good = [
    [
        'batch-1',
        [ '2020-10-06-08-22-dzdsdu', '2020-10-06-05-04-hcadfm', '2020-10-06-07-56-uvdsrj'],
    ],
    [
        'batch-2',
        ['2020-10-06-21-10-zygkme', '2020-10-06-20-25-exwcgm', '2020-10-06-20-15-dugvqd',
         '2020-10-06-19-29-oxxoix', '2020-10-06-19-26-wmlfjf', '2020-10-06-19-19-wwswcs',
         '2020-10-06-19-08-hlflgw', '2020-10-06-18-33-hinvyo', '2020-10-06-18-13-nhchsg',
         '2020-10-06-17-24-blhrnd']
    ],
    [
        'batch-3',
        ['2020-10-08-12-03-dqfjsy', '2020-10-08-13-45-swkatk',
         '2020-10-08-06-10-cfaysi', '2020-10-08-02-23-qewwai', '2020-10-08-07-22-zjstpy',
         '2020-10-08-13-14-xjdsbx', '2020-10-08-05-22-aufwvm',
         '2020-10-08-04-47-putftz', '2020-10-08-06-42-idzkjw', '2020-10-08-02-20-rofyii', 
         '2020-10-08-11-08-yaeiyp', '2020-10-08-12-15-jancmb', '2020-10-08-04-04-kqflhs',
         '2020-10-08-04-14-ntmrkr', '2020-10-08-00-53-rvhbhj', '2020-10-08-15-22-irpgmi',
         '2020-10-08-13-29-spthzg', '2020-10-08-07-11-tzvxyl', '2020-10-08-10-11-yzplzk',
         '2020-10-08-15-48-bgfdgh', '2020-10-08-14-56-mwxfvg', '2020-10-08-10-36-plldjc',
         '2020-10-08-09-41-mdiyof', '2020-10-08-14-21-sqdmbi', '2020-10-08-02-49-ufinbn',
         '2020-10-08-12-44-xihhcm', '2020-10-08-12-26-tpezky', '2020-10-08-01-54-rhxgkt',
         '2020-10-08-09-34-bemcbu', '2020-10-08-04-10-pjngma', 
         '2020-10-08-14-59-jfmdyu', '2020-10-08-05-27-vaqlcs', '2020-10-08-01-56-wwuycr',
         '2020-10-08-01-52-axeqoz', '2020-10-08-08-13-qaxbta', '2020-10-08-01-36-najvrz',
         '2020-10-08-15-02-mrmxzr', '2020-10-08-03-31-zxexrp', '2020-10-08-03-47-kugzvx']
    ]
]

In [ ]:
for group_dir, dirnames in good:
    for dirname in dirnames:
        
        key = dirname.split('-')[-1]
        #if key != 'hcadfm':
        #    continue
        print(dirname)
        path = f'./results/{group_dir}/{dirname}'
        
        fast_vid = f'{path}/{key}_fast.mp4'
        if not os.path.exists(fast_vid):
            !ffmpeg -y -i {path}/{key}.mp4 -filter:v "setpts=0.16666666666*PTS" {fast_vid}
        
        fast_small_vid = f'{path}/{key}_fast_small.mp4'
        if not os.path.exists(fast_small_vid):
            !ffmpeg -y -i {path}/{key}.mp4 -filter:v "setpts=PTS/6,scale=256:-1" {fast_small_vid}
                
        small_img = f'{path}/{key}_small.jpg'
        if not os.path.exists(small_img):
            !convert {path}/{key}.jpg -resize 350x350 {small_img}
        
        !ffmpeg -y -i {path}/{key}.mp4 -frames:v 1 -vf scale=350:-2 -q:v 3  {path}/{key}_frame0.jpg
        
        s3_path = f's3://derivative-works/results/{group_dir}/{dirname}'
        !aws s3 sync ./results/{group_dir}/{dirname} {s3_path} --acl public-read --quiet
        print('-'*80)
#         break

# Create data json

In [ ]:
import json
data = []
for group_dir, dirnames in good:
    for dirname in dirnames:
        key = dirname.split('-')[-1]
        path = f'./results/{group_dir}/{dirname}'
        with open(path+'/image_names.txt', 'r') as f:
            image_names = json.load(f)
        data.append({
            'key': key,
            'path': f'{group_dir}/{dirname}',
            'palette_keys': [s.split('/')[-1].split('_hires')[0] for s in image_names]
        })


In [ ]:
data